In [1]:
!pip install conllu

In [2]:
!pip install pyconll

In [3]:
import pandas as pd
import pyconll
from conllu import parse
import os
from tqdm import tqdm
from collections import Counter

In [ ]:
full = [] # all conllu texts
all_sources = [] #data paths

Load SinTagRus

In [ ]:
train_a = pyconll.load_from_file('Syntagrus/ru_syntagrus-ud-train-a.conllu')
train_b = pyconll.load_from_file('Syntagrus/ru_syntagrus-ud-train-b.conllu')
test = pyconll.load_from_file('Syntagrus/ru_syntagrus-ud-test.conllu')
dev = pyconll.load_from_file('Syntagrus/ru_syntagrus-ud-dev.conllu')

In [ ]:
'''for filename in tqdm(os.listdir('./Taiga/tagged_texts')):
    with open(os.path.join('./Taiga/tagged_texts', filename), 'r', encoding="utf8", errors='ignore') as f:
        text = f.read()
        text = text.split('\n\n')
        for i in text:
            if 'sent_id' in i:
                full.append(pyconll.load_from_string(i)[0])'''

Taiga small files

In [ ]:
for root, dirs, files in tqdm(os.walk("./Taiga_small", topdown=False)):
    for name in tqdm(files):
        with open(os.path.join(root, name), 'r', encoding="utf8", errors='ignore') as f:
            print(os.path.join(root, name))
            all_sources.append(os.path.join(root, name))
            text = f.read()
            text = text.split('\n\n')
            for i in text:
                if 'sent_id' in i:
                    full.append(pyconll.load_from_string(i)[0])

In [ ]:
for i in train_a:
    full.append(i)
    all_sources.append('SynTagRus')
for i in train_b:
    full.append(i)
    all_sources.append('SynTagRus')
for i in test:
    full.append(i)
    all_sources.append('SynTagRus')
for i in dev:
    full.append(i)
    all_sources.append('SynTagRus')

In [ ]:
sentence_texts = []
sentence_nums = []
token_forms = []
token_lemmas = []
token_upos = []
token_cases = []
token_animacy = []
token_gender = []
token_number = []
token_heads = []

for i, sent in enumerate(tqdm(full)):
    for j, token in enumerate(sent):
        if token.deprel=='obj':
            sentence_texts.append(sent.text)
            token_forms.append(token.form)
            token_lemmas.append(token.lemma)
            token_upos.append(token.upos)
            try: 
                token_cases.append(list(token.feats['Case'])[0])
            except KeyError:
                token_cases.append(None)
            try: 
                token_animacy.append(list(token.feats['Animacy'])[0])
            except KeyError:
                token_animacy.append(None)
            try: 
                token_gender.append(list(token.feats['Gender'])[0])
            except KeyError:
                token_gender.append(None)
            try: 
                token_number.append(list(token.feats['Number'])[0])
            except KeyError:
                token_number.append(None)
            token_heads.append(token.head)
            sentence_nums.append(i)

In [ ]:
verb_forms = []
verb_lemmas = []
verb_aspects = []
verb_forms_char = []

for i, num in enumerate(tqdm(sentence_nums)):
    ids = []
    for token in full[num]:
        ids.append(token.id)
    if token_heads[i] not in ids:
        verb_forms.append(None)
        verb_lemmas.append(None)
        verb_aspects.append(None)
        verb_forms_char.append(None)
    for token in full[num]:
        if token.id == token_heads[i] and token.upos == 'VERB':
            if 'Case' not in token.feats.keys():
                verb_forms.append(token.form)
                verb_lemmas.append(token.lemma)
                if 'Aspect' in token.feats.keys():
                    verb_aspects.append(list(token.feats['Aspect'])[0])
                else:
                    verb_aspects.append(None)
                if 'VerbForm' in token.feats.keys():
                    verb_forms_char.append(list(token.feats['VerbForm'])[0])
                else:
                    verb_forms_char.append(None)
            elif 'Case' in token.feats.keys():
                verb_forms.append(None)
                verb_lemmas.append(None) #delete parts
                verb_aspects.append(None)
                verb_forms_char.append(None)
            else:
                verb_forms.append(None) #dropna
                verb_lemmas.append(None)        
                verb_aspects.append(None)
                verb_forms_char.append(None)

In [ ]:
data = {'SentenceText': sentence_texts, 'SentenceNum': sentence_nums,
        'VerbForm': verb_forms, 'VerbLemma': verb_lemmas, 
        'VerbAspect': verb_aspects, 'VerbFormChar': verb_forms_char,
        'ObjectForm': token_forms, 'ObjectLemma': token_lemmas,
        'ObjectCase': token_cases, 'ObjectPOS': token_upos,
        'ObjectAnimacy': token_animacy, 'ObjectGender': token_gender,
        'ObjectNumber': token_number}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df

## Создание файла с итоговым датасетом

In [ ]:
df.to_csv('DO_Var_Data.csv', index=False)

## Чтение файла с итоговым датасетом

In [4]:
df2 = pd.read_csv('../input/do-var-data/DO_Var_Data.csv')

In [5]:
df2

### Разметка Nom вместо Acc у прямых дополнений

In [6]:
df2[df2['ObjectCase'] == 'Nom']['SentenceText'].tolist()[:5]

### Очистка данных от лишних символов и приведение к нижнему регистру

In [7]:
columns = ['VerbForm', 'VerbLemma', 'ObjectForm', 'ObjectLemma']
cyr = ["а","б","в","г","д","е","ё","ж","з","и","й","к","л","м","н","о",
       "п","р","с","т","у","ф","х","ц","ч","ш","щ","ъ","ы","ь","э","ю","я"]

In [8]:
def cyrillic(df, column):
    clean_words = []
    for i in df[column]:
        for symbol in i:
            if symbol not in cyr:
                i = i.replace(symbol, '')
            else:
                i = i
        clean_words.append(i)
    return clean_words

In [9]:
for column in tqdm(columns):
    df2[column] = df2[column].str.lower()
    df2[column] = cyrillic(df2, column)

### Частотность встречаемых падежей

In [10]:
df2.ObjectCase.value_counts()

### Очищенный DataFrame

In [11]:
df2

### Графики соотношений количества встречаемых падежей

In [12]:
df3 = df2[df2['ObjectCase'] != 'Acc'].ObjectCase.value_counts().plot.bar()

In [13]:
df4 = df2[df2['ObjectCase'] != 'Acc']
df5 = df4[df4['ObjectCase'] != 'Nom'].ObjectCase.value_counts().plot.bar()

### DataFrame без дополнений в Nom

In [14]:
df_non_nom = df2[df2['ObjectCase'] != 'Nom']

In [15]:
df_non_nom

## Объеднинение по лемме глагола

In [16]:
df6 = df_non_nom[['VerbLemma', 'ObjectLemma', 'ObjectCase']].groupby(['VerbLemma', 'ObjectLemma', 'ObjectCase'])['ObjectCase'].count()

In [17]:
df6

In [18]:
df6['id'] = df6.index.values

In [19]:
df6 = pd.DataFrame(df6)

In [20]:
df6

In [21]:
df7 = df6.to_dict(orient='dict') 

In [22]:
phrases = []

for i in tqdm(list(df7['ObjectCase'].keys())):
    phrases.append(i[0] + ' ' + i[1])

phrases_stat = Counter(phrases)

In [23]:
variable_phrases = [] #list of lists with verb + obj constructions

for key in tqdm(list(phrases_stat.keys())):
    if phrases_stat[key] >= 2:
        variable_phrases.append(key)

In [24]:
indexes = []

for i, j in enumerate(tqdm(df7['ObjectCase'])):
    phrase = j[0] + ' ' + j[1]
    if phrase in variable_phrases:
        indexes.append(i)

In [25]:
verb_lemma_2 = []
do_lemma_2 = []
case = []
freq_2 = list(df7['ObjectCase'].values())

for i in df7['ObjectCase']:
    verb_lemma_2.append(i[0])
    do_lemma_2.append(i[1])
    case.append(i[2])

In [26]:
data2 = {'VerbLemma': verb_lemma_2, 'DOLemma': do_lemma_2,
        'Case': case, 'Frequency': freq_2}

In [27]:
df8 = pd.DataFrame(data2)

In [28]:
df9 = df8.iloc[indexes].reset_index(drop=True)

In [29]:
df9['Frequency'] = df9['Frequency'].apply(str)

In [30]:
df10 = df9.groupby(['VerbLemma', 'DOLemma'], as_index=False).agg({'Case': ' '.join, 'Frequency': ' '.join})

In [31]:
df10

In [32]:
set(df10['Case'].tolist())

### Acc vs. Dat

In [33]:
df10[df10['Case'] == 'Acc Dat']

**Направлять власть**

In [34]:
df_non_nom[(df_non_nom['VerbLemma'] == 'направлять') & (df_non_nom['ObjectLemma'] == 'власть')]

In [35]:
df_non_nom[(df_non_nom['VerbLemma'] == 'направлять') & (df_non_nom['ObjectLemma'] == 'власть')]['SentenceText'].tolist()

### Acc vs. Gen

In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df10[df10['Case'] == 'Acc Gen'])

In [37]:
len(df10[df10['Case'] == 'Acc Gen'])

**Стоить тысяч**

In [38]:
df_non_nom[(df_non_nom['VerbLemma'] == 'стоить') & (df_non_nom['ObjectLemma'] == 'тысяча')]

In [39]:
df_non_nom[(df_non_nom['VerbLemma'] == 'стоить') & (df_non_nom['ObjectLemma'] == 'тысяча')]['SentenceText'].tolist()

**Требовать рассмотрение**

In [40]:
df_non_nom[(df_non_nom['VerbLemma'] == 'требовать') & (df_non_nom['ObjectLemma'] == 'рассмотрение')]

In [41]:
df_non_nom[(df_non_nom['VerbLemma'] == 'требовать') & (df_non_nom['ObjectLemma'] == 'рассмотрение')]['SentenceText'].tolist()

**Стоить копейка**

In [42]:
df_non_nom[(df_non_nom['VerbLemma'] == 'стоить') & (df_non_nom['ObjectLemma'] == 'копейка')]

In [43]:
df_non_nom[(df_non_nom['VerbLemma'] == 'стоить') & (df_non_nom['ObjectLemma'] == 'копейка')]['SentenceText'].tolist()

**Извлечь сведений**

In [44]:
df_non_nom[(df_non_nom['VerbLemma'] == 'извлечь') & (df_non_nom['ObjectLemma'] == 'сведение')]

In [45]:
df_non_nom[(df_non_nom['VerbLemma'] == 'извлечь') & (df_non_nom['ObjectLemma'] == 'сведение')]['SentenceText'].tolist()

**Хотеть преимуществ**

In [46]:
df_non_nom[(df_non_nom['VerbLemma'] == 'хотеть') & (df_non_nom['ObjectLemma'] == 'преимущество')]

In [47]:
df_non_nom[(df_non_nom['VerbLemma'] == 'хотеть') & (df_non_nom['ObjectLemma'] == 'преимущество')]['SentenceText'].tolist()

**Платить налог**

In [48]:
df_non_nom[(df_non_nom['VerbLemma'] == 'платить') & (df_non_nom['ObjectLemma'] == 'налог')]

In [49]:
df_non_nom[(df_non_nom['VerbLemma'] == 'платить') & (df_non_nom['ObjectLemma'] == 'налог') & (df_non_nom['ObjectCase'] == 'Gen')]['SentenceText'].tolist()

In [50]:
df_non_nom[(df_non_nom['VerbLemma'] == 'платить') & (df_non_nom['ObjectLemma'] == 'налог') & (df_non_nom['ObjectCase'] == 'Acc')]['SentenceText'].tolist()[:3]

Здесь можно наблюдать вариабельность Acc vs. Gen

Вариант написания "не платят прямые налоги" также является грамматичным.

**Вызывать вопрос**

In [51]:
df_non_nom[(df_non_nom['VerbLemma'] == 'вызывать') & (df_non_nom['ObjectLemma'] == 'вопрос')]

In [52]:
df_non_nom[(df_non_nom['VerbLemma'] == 'вызывать') & (df_non_nom['ObjectLemma'] == 'вопрос')]['SentenceText'].tolist()

Это тоже вызывает много вопросов.

Однако существующая сегодня процедура выборов вызывает немало серьезных вопросов, предупреждают эксперты.

Влияние числительного на падеж прямого дополнения.

**Составлять процент**

In [53]:
df_non_nom[(df_non_nom['VerbLemma'] == 'составлять') & (df_non_nom['ObjectLemma'] == 'процент')]

In [54]:
df_non_nom[(df_non_nom['VerbLemma'] == 'составлять') & (df_non_nom['ObjectLemma'] == 'процент')]['SentenceText'].tolist()

В Петербурге, где отраслевая наука была очень сильной и составляла значительный процент, по численности работающих она упала раза в три.

Интернет утверждает: азиаты и африканцы составляют 80 процентов населения Земли.

Влияние числительного на падеж прямого дополнения.

**Читать роман**

In [55]:
df_non_nom[(df_non_nom['VerbLemma'] == 'читать') & (df_non_nom['ObjectLemma'] == 'роман')]

In [56]:
df_non_nom[(df_non_nom['VerbLemma'] == 'читать') & (df_non_nom['ObjectLemma'] == 'роман')]['SentenceText'].tolist()

вариативность Acc vs. Gen

**Возводить комплекс**

In [57]:
df_non_nom[(df_non_nom['VerbLemma'] == 'возводить') & (df_non_nom['ObjectLemma'] == 'комплекс')]

In [58]:
df_non_nom[(df_non_nom['VerbLemma'] == 'возводить') & (df_non_nom['ObjectLemma'] == 'комплекс')]['SentenceText'].tolist()

'ГК "Город" возводит в Петербурге три жилых комплекса, насчитывающих около 5 тысяч дольщиков.'

Влияние числительного на падеж прямого дополнения.

### Acc vs. Ins

In [59]:
df10[df10['Case'] == 'Acc Ins']

**Разводить рука**

In [60]:
df_non_nom[(df_non_nom['VerbLemma'] == 'разводить') & (df_non_nom['ObjectLemma'] == 'рука')]

In [61]:
df_non_nom[(df_non_nom['VerbLemma'] == 'разводить') & (df_non_nom['ObjectLemma'] == 'рука')]['SentenceText'].tolist()

### Acc vs. Loc

In [62]:
df10[df10['Case'] == 'Acc Loc']

**Делать сила**

In [63]:
df_non_nom[(df_non_nom['VerbLemma'] == 'делать') & (df_non_nom['ObjectLemma'] == 'сила')]

In [64]:
df_non_nom[(df_non_nom['VerbLemma'] == 'делать') & (df_non_nom['ObjectLemma'] == 'сила')]['SentenceText'].tolist()

Предлог + лексикализованное выражение.

**Ездить троллейбус**

In [65]:
df_non_nom[(df_non_nom['VerbLemma'] == 'ездить') & (df_non_nom['ObjectLemma'] == 'троллейбус')]

In [66]:
df_non_nom[(df_non_nom['VerbLemma'] == 'ездить') & (df_non_nom['ObjectLemma'] == 'троллейбус')]['SentenceText'].tolist()

Предлог.

### Acc vs. Par

In [67]:
df10[df10['Case'] == 'Acc Par']

**Выпать чай**

In [68]:
df_non_nom[(df_non_nom['VerbLemma'] == 'выпать') & (df_non_nom['ObjectLemma'] == 'чай')]

In [69]:
df_non_nom[(df_non_nom['VerbLemma'] == 'выпать') & (df_non_nom['ObjectLemma'] == 'чай')]['SentenceText'].tolist()

С деепричастием употтреблняется партитив.

**Пить чай**

In [70]:
df_non_nom[(df_non_nom['VerbLemma'] == 'пить') & (df_non_nom['ObjectLemma'] == 'чай')]

In [71]:
df_non_nom[(df_non_nom['VerbLemma'] == 'пить') & (df_non_nom['ObjectLemma'] == 'чай' ) & (df_non_nom['ObjectCase'] == 'Acc' )]['SentenceText'].tolist()[:5]

In [72]:
df_non_nom[(df_non_nom['VerbLemma'] == 'пить') & (df_non_nom['ObjectLemma'] == 'чай' ) & (df_non_nom['ObjectCase'] == 'Par' )]['SentenceText'].tolist()

Деепричастие.

Прямое дополнение перед глаголом.

**Голосовать толк**

In [73]:
df_non_nom[(df_non_nom['VerbLemma'] == 'голосовать') & (df_non_nom['ObjectLemma'] == 'толк' )]

In [74]:
df_non_nom[(df_non_nom['VerbLemma'] == 'голосовать') & (df_non_nom['ObjectLemma'] == 'толк' )]['SentenceText'].tolist()

Лексикализация.

**Делать раз**

In [75]:
df_non_nom[(df_non_nom['VerbLemma'] == 'делать') & (df_non_nom['ObjectLemma'] == 'раз' )]

In [76]:
df_non_nom[(df_non_nom['VerbLemma'] == 'делать') & (df_non_nom['ObjectLemma'] == 'раз' )]['SentenceText'].tolist()

Отрицание перед прямым дополнением.

Лексикализация.

### Acc vs. Voc

In [77]:
df10[df10['Case'] == 'Acc Voc']

**Давать бог**

In [78]:
df_non_nom[(df_non_nom['VerbLemma'] == 'давать') & (df_non_nom['ObjectLemma'] == 'бог')]

In [79]:
df_non_nom[(df_non_nom['VerbLemma'] == 'давать') & (df_non_nom['ObjectLemma'] == 'бог')]['SentenceText'].tolist()

**Привести господь**

In [80]:
df_non_nom[(df_non_nom['VerbLemma'] == 'привести') & (df_non_nom['ObjectLemma'] == 'господь')]

In [81]:
df_non_nom[(df_non_nom['VerbLemma'] == 'привести') & (df_non_nom['ObjectLemma'] == 'господь')]['SentenceText'].tolist()

## Объединение по форме глагола

In [82]:
df11 = df_non_nom[['VerbForm', 'ObjectLemma', 'ObjectCase']].groupby(['VerbForm', 'ObjectLemma', 'ObjectCase'])['ObjectCase'].count()

In [83]:
df11

In [84]:
df11['id'] = df11.index.values

In [85]:
df11 = pd.DataFrame(df11)

In [86]:
df11

In [87]:
df12 = df11.to_dict(orient='dict') 

In [88]:
phrases2 = []

for i in tqdm(list(df12['ObjectCase'].keys())):
    phrases2.append(i[0] + ' ' + i[1])

phrases_stat2 = Counter(phrases2)

In [89]:
variable_phrases2 = [] #list of lists with verb + obj constructions

for key in tqdm(list(phrases_stat2.keys())):
    if phrases_stat2[key] >= 2:
        variable_phrases2.append(key)

In [90]:
indexes2 = []

for i, j in enumerate(tqdm(df12['ObjectCase'])):
    phrase2 = j[0] + ' ' + j[1]
    if phrase2 in variable_phrases2:
        indexes2.append(i)

In [91]:
verb_form_3 = []
do_lemma_3 = []
case2 = []
freq_3 = list(df12['ObjectCase'].values())

for i in df12['ObjectCase']:
    verb_form_3.append(i[0])
    do_lemma_3.append(i[1])
    case2.append(i[2])

In [92]:
data3 = {'VerbForm': verb_form_3, 'DOLemma': do_lemma_3,
        'Case': case2, 'Frequency': freq_3}

In [93]:
df13 = pd.DataFrame(data3)

In [94]:
df14 = df13.iloc[indexes2].reset_index(drop=True)

In [95]:
df14['Frequency'] = df14['Frequency'].apply(str)

In [96]:
df15 = df14.groupby(['VerbForm', 'DOLemma'], as_index=False).agg({'Case': ' '.join, 'Frequency': ' '.join})

In [97]:
df15

In [98]:
set(df15['Case'].tolist())

### Acc vs. Dat

In [99]:
df15[df15['Case'] == 'Acc Dat']

**Пойти жертва**

In [100]:
df_non_nom[(df_non_nom['VerbForm'] == 'пойти') & (df_non_nom['ObjectLemma'] == 'жертва')]

In [101]:
df_non_nom[(df_non_nom['VerbForm'] == 'пойти') & (df_non_nom['ObjectLemma'] == 'жертва')]['SentenceText'].tolist()

В данном случае в предложении перед прямым дополненеим расположен предлог, что не соответствует нашим требованиям к рассмтариваемой в исследовании конструкции.

### Acc vs. Gen

In [102]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df15[df15['Case'] == 'Acc Gen'])

In [103]:
len(df15[df15['Case'] == 'Acc Gen'])

**Выдержать срок**

In [104]:
df_non_nom[(df_non_nom['VerbForm'] == 'выдержать') & (df_non_nom['ObjectLemma'] == 'срок')]

In [105]:
df_non_nom[(df_non_nom['VerbForm'] == 'выдержать') & (df_non_nom['ObjectLemma'] == 'срок')]['SentenceText'].tolist()

**Благодарит партнер**

In [106]:
df_non_nom[(df_non_nom['VerbForm'] == 'благодарит') & (df_non_nom['ObjectLemma'] == 'партнер')]

In [107]:
df_non_nom[(df_non_nom['VerbForm'] == 'благодарит') & (df_non_nom['ObjectLemma'] == 'партнер')]['SentenceText'].tolist()

Управление прилагательного прямым дополнением (ограничения на падежную форму).

**Арестовал фигуранта**

In [108]:
df_non_nom[(df_non_nom['VerbForm'] == 'арестовал') & (df_non_nom['ObjectLemma'] == 'фигурант')]

In [109]:
df_non_nom[(df_non_nom['VerbForm'] == 'арестовал') & (df_non_nom['ObjectLemma'] == 'фигурант')]['SentenceText'].tolist()

### Acc vs. Ins

In [110]:
df15[df15['Case'] == 'Acc Ins']

**Привела лицо**

In [111]:
df_non_nom[(df_non_nom['VerbForm'] == 'привела') & (df_non_nom['ObjectLemma'] == 'лицо')]

In [112]:
df_non_nom[(df_non_nom['VerbForm'] == 'привела') & (df_non_nom['ObjectLemma'] == 'лицо')]['SentenceText'].tolist()

### Acc vs. Loc

In [113]:
df15[df15['Case'] == 'Acc Loc']

**Написать глава**

In [114]:
df_non_nom[(df_non_nom['VerbForm'] == 'написать') & (df_non_nom['ObjectLemma'] == 'глава')]

In [115]:
df_non_nom[(df_non_nom['VerbForm'] == 'написать') & (df_non_nom['ObjectLemma'] == 'глава')]['SentenceText'].tolist()

Омонимия. 

### Acc vs. Par

In [116]:
df15[df15['Case'] == 'Acc Par']

### Acc vs. Voc

In [117]:
df15[df15['Case'] == 'Acc Voc']